# Clases normales en Python vs Pydantic

En este notebook vamos a ver:

1. Cómo definir clases normales en Python para representar datos.
2. Qué problema tienen cuando recibimos datos "malos".
3. Cómo Pydantic nos ayuda a validar y transformar datos automáticamente.

In [6]:
class Usuario:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

    def saludar(self):
        return f"Hola, soy {self.nombre} y tengo {self.edad} años."


usuario_ok = Usuario("Ana", 25)
print(usuario_ok.saludar())
print("Tipo de edad:", type(usuario_ok.edad))

Hola, soy Ana y tengo 25 años.
Tipo de edad: <class 'int'>


In [7]:
usuario_raro = Usuario("Pepe", "veintitrés")

print(usuario_raro.saludar())
print("Tipo de edad:", type(usuario_raro.edad))


Hola, soy Pepe y tengo veintitrés años.
Tipo de edad: <class 'str'>


In [9]:
class UsuarioValidado:
    def __init__(self, nombre, edad):
        if not isinstance(nombre, str):
            raise TypeError("nombre debe ser str")
        if not isinstance(edad, int):
            raise TypeError("edad debe ser int")

        self.nombre = nombre
        self.edad = edad

    def saludar(self):
        return f"Hola, soy {self.nombre} y tengo {self.edad} años."

usuario_bien = UsuarioValidado("Lucía", 30)
print(usuario_bien.saludar())

usuario_mal = UsuarioValidado("Mario", "treinta")

Hola, soy Lucía y tengo 30 años.


TypeError: edad debe ser int

## Ahora entra Pydantic

Pydantic nos permite:

- Definir modelos de datos con tipos (como en las clases).
- Validar automáticamente que los datos cumplen esos tipos.
- Convertir tipos cuando es posible (por ejemplo, "25" → 25).
- Dar errores claros cuando los datos son inválidos.

In [11]:
from pydantic import BaseModel

class UsuarioPydantic(BaseModel):
    nombre: str
    edad: int


usuario_ok = UsuarioPydantic(nombre="Ana", edad=25)
print(usuario_ok)
print("Tipo de edad:", type(usuario_ok.edad))

nombre='Ana' edad=25
Tipo de edad: <class 'int'>


In [12]:
usuario_convertido = UsuarioPydantic(nombre="Carlos", edad="28")

print(usuario_convertido)
print("Tipo de edad:", type(usuario_convertido.edad))

nombre='Carlos' edad=28
Tipo de edad: <class 'int'>


In [13]:
from pydantic import ValidationError

try:
    # Esto ya no tiene sentido: edad = "veintiocho" no se puede convertir a int
    usuario_error = UsuarioPydantic(nombre="Pepe", edad="veintiocho")
except ValidationError as e:
    print("Error de validación de Pydantic:")
    print(e)

Error de validación de Pydantic:
1 validation error for UsuarioPydantic
edad
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='veintiocho', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [14]:
from typing import Optional


class UsuarioConEmail(BaseModel):
    nombre: str
    edad: int
    email: Optional[str] = None  # Campo opcional con valor por defecto

usuario_sin_email = UsuarioConEmail(nombre="Laura", edad=22)
usuario_con_email = UsuarioConEmail(nombre="David", edad=35, email="david@example.com")

print("Sin email:", usuario_sin_email)
print("Con email:", usuario_con_email)

Sin email: nombre='Laura' edad=22 email=None
Con email: nombre='David' edad=35 email='david@example.com'


In [15]:
class Direccion(BaseModel):
    ciudad: str
    pais: str


class UsuarioConDireccion(BaseModel):
    nombre: str
    edad: int
    direccion: Direccion


datos = {
    "nombre": "Sofía",
    "edad": 29,
    "direccion": {
        "ciudad": "Málaga",
        "pais": "España",
    },
}

usuario_con_direccion = UsuarioConDireccion(**datos)
print(usuario_con_direccion)
print("Ciudad:", usuario_con_direccion.direccion.ciudad)

nombre='Sofía' edad=29 direccion=Direccion(ciudad='Málaga', pais='España')
Ciudad: Málaga
